In [1]:
import sys
import os

ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(os.path.abspath(ROOT))

In [2]:
import json
import os
import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt
import k3d

from config import ROS_DATASET_PATH

# Set dataset and output paths
dataset_root = ROS_DATASET_PATH

In [3]:
# Configuration
rosbag_id = "ros2bag_20250331_092443"
meta_data_path = f'{dataset_root}/{rosbag_id}/metadata.json'
bag_path = f'{dataset_root}/{rosbag_id}'
frame = 10

# Load metadata
with open(meta_data_path, 'r') as f:
    metadata = json.load(f)

def load_frame(sensor_type, frame_index=0):
    """Load a specific frame's data with proper validation"""
    frame_info = metadata[sensor_type][frame_index]
    file_path = os.path.join(bag_path, frame_info['filename'])
    
    try:
        with open(file_path, 'rb') as f:
            # Read header
            point_count = np.fromfile(f, dtype='<u4', count=1)[0]
            has_velocity = np.fromfile(f, dtype='u1', count=1)[0]
            
            # Read points
            points = np.fromfile(f, dtype='<f4', count=point_count*3)
            points = points.reshape(-1, 3)
            
            if sensor_type == 'radar' and has_velocity:
                velocities = np.fromfile(f, dtype='<f4', count=point_count)
                return points, velocities.flatten(), frame_info  # Changed to 1D array
            else:
                return points, None, frame_info
                
    except Exception as e:
        print(f"Error loading {sensor_type} frame {frame_index}: {str(e)}")
        return np.zeros((0, 3)), None, frame_info

def print_points(points, velocities=None, title="Point Cloud", max_points=5):
    """Print first few points in a readable format"""
    print(f"\n{title} (first {min(max_points, len(points))} of {len(points)} points):")
    for i in range(min(max_points, len(points))):
        point_str = f"Point {i+1}: x={points[i,0]:.3f}, y={points[i,1]:.3f}, z={points[i,2]:.3f}"
        if velocities is not None:
            # Handle both 1D and 2D velocity arrays
            vel = velocities[i] if isinstance(velocities, np.ndarray) else velocities
            point_str += f", velocity={float(vel):.3f} m/s"
        print(point_str)

# Load and display radar data
radar_points, radar_velocities, radar_info = load_frame('radar', frame)
print_points(radar_points, radar_velocities, 
             f"Radar Point Cloud (Frame {radar_info['frame_count']}, Timestamp: {radar_info['timestamp']})")

# Load and display lidar data
lidar_points, _, lidar_info = load_frame('lidar', frame)
print_points(lidar_points, None, 
             f"Lidar Point Cloud (Frame {lidar_info['frame_count']}, Timestamp: {lidar_info['timestamp']})")


Radar Point Cloud (Frame 10, Timestamp: 1743405884458642729) (first 5 of 162 points):
Point 1: x=14.312, y=-0.537, z=3.122, velocity=0.014 m/s
Point 2: x=15.132, y=-1.347, z=2.871, velocity=-0.011 m/s
Point 3: x=16.248, y=-2.208, z=1.666, velocity=-0.033 m/s
Point 4: x=16.187, y=-2.208, z=2.180, velocity=-0.033 m/s
Point 5: x=16.603, y=-2.349, z=1.910, velocity=0.014 m/s

Lidar Point Cloud (Frame 10, Timestamp: 1743405885862791741) (first 5 of 28960 points):
Point 1: x=29.404, y=-0.000, z=-7.863
Point 2: x=29.386, y=-0.005, z=-6.776
Point 3: x=29.434, y=-0.010, z=-5.709
Point 4: x=29.679, y=-0.016, z=-4.695
Point 5: x=29.939, y=-0.021, z=-3.671


In [4]:
auto_frame = True
grid_visible = False

pc_radar = radar_points

plot = k3d.plot(camera_auto_fit=auto_frame, axes_helper=0.0, grid_visible=grid_visible)

#Radar-Points:
plot += k3d.points(positions=np.asarray(pc_radar[:, :3], dtype=float), point_size=0.5)


plot.display()

c:\Users\leopo\anaconda3\envs\demons-env\lib\site-packages\traittypes\traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()